# Welcome to DES application coding process
First we need to provide the input of the 64 bits as 8 characters:
- Plaintext = plaintext
- Key = key

In [38]:
plaintext = "Ineedyou"
Key = "LoveLong"

Convert the plaintext and key variable into Binary string

In [39]:
def str2bin(text):
    binary = ''.join(format(ord(i), '08b') for i in text)
    return binary

In [40]:
# Testing
plaintext_bin = str2bin(plaintext)
key_bin = str2bin(Key)
print("Plaintext in binary: ", plaintext_bin)
print("Key in binary: ", key_bin)

Plaintext in binary:  0100100101101110011001010110010101100100011110010110111101110101
Key in binary:  0100110001101111011101100110010101001100011011110110111001100111


But we also need a function to convert binary to string after the 16 rounds of encryption output which is a binary string.

In [41]:
def bin2str(binary):
    return ''.join(chr(int(binary[i:i+8], 2)) for i in range(0, len(binary), 8))

Since we need to perform the permutation a lots which `Output[i] = Input[table[i]]`

In [42]:
def permute(Input, table):
    return [Input[i-1] for i in table]

And We need to create some utility that will help us to generate random array (input) to perform permutation which we going to use a lots

In [43]:
import random

# Function generate the unique random array
def generate_unique_random_array(size, start, end):
    array =  random.sample(range(start, end), size)
    return array

# Function generate the random array with duplicate elements 
# it will be used for expansion box 
def generate_random_table(start, end, size, max_occurrences):
    random_numbers = random.sample(max_occurrences* list(range(start, end)), size)
    return random_numbers

# Function generate matrix of 2 dimensions array with random values 
# it will be used for generating the S-boxes
def create_matrix(rows, cols, start, end):
    return [[random.randint(start, end) for _ in range(cols)] for _ in range(rows)]

## Key Generation / Operation

In order to perform Key Generation, We will break down the main function into 2 parts
- Parity drop
- Round-key Generator

### Perform Parity Drop 
1. we need to Parity Drop table which have 56-bits size randomly generate with unique value selected from [1, 64]: PD_table
2. We will perform Key permutation of 64-bits to 56-bits with Parity Drop table which given use the result of the Parity Drop Box: PD_permute_key -> PD_box

In [44]:
def PD_box(key_bin):
    # we input the key of 64-bits but we use only 56-bits so we need to generate a random array of 56 unique numbers from 1 to 64 for table for Permutation
    PD_table = generate_unique_random_array(56, 1, 65)
    # Key permutation 64-bit to 56-bit
    PD_permute_key = permute(key_bin, PD_table)
    # return the cipher key of 56-bits
    return PD_permute_key

### Round-Key Generator
It Create 16 keys of the 48-bits of 56-bits cipher key
So we need to perform 16 rounds
 - we need to slip the cipher keys into 28-bits equal side Left and Right

In [45]:
# First we need to split the cipher key into two parts
def split_array(arr, size):
    left = key_bin[:size]
    right = key_bin[size:]
    return left, right

In Key rotation

We break down the each round into 3 steps
 - Shift left of left and right key
 - Concatenate both side
 - Compression Drop-box 

In [46]:
# Function Shift left
def shift_left(arr_bin, shift):
    return arr_bin[shift:] + arr_bin[:shift]

# Function Round-key generator
def keys_generator(key_bin):
    # store the generated keys in a list
    generate_keys = []
    # generate compressed-Dbox table from 1-56 key to 48-bit key length
    # we need to use the same compressed_Dbox_table for all rounds of the key generation
    compressed_Dbox_table = generate_unique_random_array(48, 1, 57)
    # we need to split the 56-bit key into two 28-bit keys
    left, right = split_array(key_bin, 28)
    # shift our bit to the left
    for i in range(1, 17):
        # set the shift value for the key rotation 
        shift = 1 if i in [1, 2, 9, 16] else 2
        # shift the left and right keys
        left = shift_left(left, shift)
        right = shift_left(right, shift)

        # This is where Concatenation of the left and right keys
        combined_output = left + right

        # compresses 56-bit key to 48-bit key
        compressed_Dbox = permute(combined_output, compressed_Dbox_table)
        generate_keys.append(compressed_Dbox) 
    return generate_keys

# Main Algorithm of DES Application
We perform
- Initial permutation -> split each plaintext into 32-bits Left, Right
- Round Function
- Final permutation

In [47]:
# Initial permutation
def ip_box(text_bin):
    ip_table = generate_unique_random_array(64, 1, 65)
    ip_permute_text = permute(text_bin,ip_table)
    return ip_permute_text

# Final permutation
def fp_box(last_round_combined_left_right_bin):
    fp_table = generate_unique_random_array(64, 1, 65)
    fp_permute_text = permute(last_round_combined_left_right_bin,fp_table)
    return fp_permute_text

# Round Function
For each round it have 2 main components
- Mixer: which we have f_Function and xor L
    - f_Function(R_(i-1),k_i)
    - xor
- Swapper:
        Left -> Right
        Right -> Left

f_function have sub 3 components
- Expansion D-box
- XOR: Key with the output of Expansion D-box
- S-boxes

In [48]:
# XOR operation of binary strings
def xor(bin1, bin2):
    # check if the binary strings are of the same length
    if len(bin1) != len(bin2):
        raise ValueError("Binary strings must have the same length.")
    return ''.join('0' if a == b else '1' for a, b in zip(bin1, bin2))

# slipt the binary string of array into chunks
def divide_chunks(input_string, chunk_size):
    return [input_string[i:i+chunk_size] for i in range(0, len(input_string), chunk_size)]

# Function F-function 
def f_function(plaintext_bin_arr_right, generate_key_bin):
    # Expansion D-box
    expansion_Dbox_table = generate_random_table(1, 33, 48, 2)
    expansion_output = permute(plaintext_bin_arr_right, expansion_Dbox_table)
    # perform XOR operation EXPANSION D-box output and the key
    xor_output = xor(expansion_output, generate_key_bin)
    # create S-boxes
    s_box = divide_chunks(xor_output, 6)
    chunk = len(s_box)

    s_box_32bits = []
    # 7 it the max index of chunk that s_box contain which have 8 chunk so -1 equal to number of the max index
    for j in range(chunk):
        s_box_table = create_matrix(4, 16, 0, 15)
        row = int(s_box[j][0] + s_box[j][5], 2)
        col = int(s_box[j][1:5], 2)
        
        bin_sbox_value = bin(s_box_table[row][col])[2:].zfill(4)
        # s_box_32bits += bin(s_box_table[row][col])[2:].zfill(4) 
        s_box_32bits.extend(list(bin_sbox_value))

    # Straight Permutation D-box
    return s_box_32bits

In [49]:
# Function Swap 
def swap(left, right):
    return right, left

# Round function - Encrypt
def round_function(left, right, key):
    # F-function 
    f_function_output = f_function(right, key)
    # Mixer
    xor_L_fR_output = xor(left, f_function_output)
    # Perform Swap
    left_new, right_new = swap(left, xor_L_fR_output)
    
    # return the new left and right
    return left_new, right_new

# DES Encryption Operation of 16 rounds

In [50]:
plaintext = "Ineedyou"
Key="LoveLong"
# convert plaintext and key to binary string
key_bin = str2bin(Key)
plaintext_bin = str2bin(plaintext)
# Initial permutation
IP_text_bin = ip_box(plaintext_bin)

# Parity drop box
PD_permute_key_output = PD_box(key_bin)    
# generate keys
generate_keys = keys_generator(PD_permute_key_output)
# split the IP_permutation into two halves
IP_left, IP_right = split_array(IP_text_bin, len(IP_text_bin)//2)

for i in range(1, 17):
    # f_function_output = f_function(IP_right, generate_keys[i-1])
    # xor_L_fR_output = xor(IP_left, f_function_output)
    # IP_left, IP_right = swap(IP_left, xor_L_fR_output)
    IP_left, IP_right = round_function(IP_left, IP_right, generate_keys[i-1])


# concatenate the left and right keys
combined_output = list(IP_left + IP_right)

# Final permutation
FP_permute_output = fp_box(combined_output)

print("Plaintext: ", plaintext)
print("Key: ", Key)
print("====================")
print("Cipher text array: ", FP_permute_output)
print("Cipher text String: ", ''.join(FP_permute_output))
print("Cipher text: ", bin2str(''.join(FP_permute_output)))

Plaintext:  Ineedyou
Key:  LoveLong
Cipher text array:  ['0', '1', '0', '1', '0', '0', '1', '1', '1', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1']
Cipher text String:  0101001111000110011110111000110111000110011100110010101100010111
Cipher text:  SÆ{Æs+
